<a href="https://colab.research.google.com/github/lc-lor-lcb/C-_MySQL_Connection/blob/master/XQ%E8%AA%B2%E9%A1%8C_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 22.8 MB/s eta 0:00:00


In [ ]:
# @title
from enum import Enum
from random import shuffle
import numpy as np
from operator import mul
import random
import termcolor

class Suit(Enum):
    SPADE = '♠'
    CLUB = '♣'
    HEART = '♡'
    DIAMOND = '♢'
    def __str__(self):
        return self.value
    def __repr__(self):
        return f"Suit.{self.name}"


class Number(Enum):
    ACE = (1, 'A')
    TWO = (2, '2')
    THREE = (3, '3')
    FOUR = (4, '4')
    FIVE = (5, '5')
    SIX = (6, '6')
    SEVEN = (7, '7')
    EIGHT = (8, '8')
    NINE = (9, '9')
    TEN = (10, '10')
    JACK = (11, 'J')
    QUEEN = (12, 'Q')
    KING = (13, 'K')

    def __init__(self, val, string):
        self.val = val
        self.string = string

    def __str__(self):
        return self.string

    def __repr__(self):
        return f"Number.{self.name}"

class Card:
    def __init__(self, suit, number):
        if not (isinstance(suit, Suit) and isinstance(number, Number)):
            raise ValueError  # Enum じゃないとエラー
        self.suit = suit
        self.number = number

    def __str__(self):
        return str(self.suit) + str(self.number)

    def __repr__(self):
        return f"Card({self.__str__()})"

    def __eq__(self, other):
        return (self.suit, self.number) == (other.suit, other.number)

    def __hash__(self):
        return hash((self.suit, self.number))


class Hand(list):
    def __init__(self,card_list):
        super().__init__(
            i for i in card_list
        )

    def check_number(self):
        number_list=[i.number.val for i in self]
        return number_list

    def check_suit(self):
        suit_list=[str(i.suit) for i in self]
        return suit_list

    def choice(self,card):
        #Card(Suit.SPADE, Number.ACE)
        if card in self:
            self.remove(card)
            return card
        else:
            raise ValueError

    def check(self,card):
        return card in self


class Deck(list):
    def __init__(self):
        super().__init__(
            Card(suit, number) for suit in Suit for number in Number
        )  # list の初期化を呼び出す
        self.shuffle()  # 最初にシャッフル
    def shuffle(self):
        shuffle(self)
    def draw(self):
        return self.pop()
    def deal(self, players_num):
        cards=[Hand(i) for i in np.array_split(self,players_num)]
        self.clear()
        return cards

In [ ]:
# @title
# ゲームの状態
class State:
    # 初期化
    def __init__(self, players_num=3,field_cards=None, players_cards=None,turn_player=None,pass_count=None,out_player=None,all_actions=None,all_players=None):
        if players_cards==None:
            deck = Deck()
            self.players_cards=deck.deal(players_num)
            self.players_num=players_num
            self.field_cards=np.zeros((4,13), dtype='int64')
            self.start_flags=[0]*self.players_num
            self.pass_count=[0]*self.players_num
            self.out_player=[]
            self.all_actions = []
            self.all_players = []
            self.all_cards=[[str(Card(suit, number))  for number in Number] for suit in Suit]
            for players_number in range(players_num):
                self.start_flags[players_number]=self.choice_seven(hand=self.players_cards[players_number])
            self.turn_player=self.start_flags.index(1)
        else:
            self.players_cards=players_cards
            self.field_cards=field_cards
            self.players_num=players_num
            self.turn_player=turn_player
            self.pass_count=pass_count
            self.out_player=out_player
            self.all_actions=all_actions
            self.all_players =all_players
            self.all_cards=[[str(Card(suit, number))  for number in Number] for suit in Suit]


    #7のカードを出す
    def choice_seven(self,hand):
        start_flag=0
        for card in [Card(suit,Number.SEVEN) for suit in Suit]:
            if hand.check(Card(Suit.DIAMOND,Number.SEVEN))==True:
                start_flag=1
            if hand.check(card)==True:
                self.put_card(hand.choice(card))
        return start_flag

    def choice_card(self,hand,card):
        hand.choice(card)

    #場にカードを出す
    def put_card(self,card):
        num=10
        for s,i in zip(Suit,range(4)):
            if card.suit==s:
                num=i
        #state.my_hands().remove(card)
        self.field_cards[num][card.number.val-1]=int(1)


    # 場で出せる手のリスト取得
    def legal_actions(self):
        actions = []
        for suit, n in zip(Suit, range(4)):
            small_side = self.field_cards[n][0:6][::-1].tolist()
            large_side = self.field_cards[n][7:13].tolist()

            # カードの場の状態をチェック
            is_ace_out = self.field_cards[n][0] == 1    # Aが出ている (インデックス 0)
            is_king_out = self.field_cards[n][12] == 1  # Kが出ている (インデックス 12)

            # --- 1. 通常の隣接カード判定 ---

            # 7より小さい側 (A-6) の最も外側に出せるカード
            if small_side.count(1) != 6:
                card_num_val = 6 - small_side.index(0)

                if card_num_val != 1:
                    actions.append(Card(suit, self.num_to_Enum(card_num_val)))

                elif self.field_cards[n][1] == 1 and not is_ace_out:
                    actions.append(Card(suit, Number.ACE))

            # 7より大きい側 (8-K) の最も外側に出せるカード
            if large_side.count(1) != 6:
                card_num_val = 8 + large_side.index(0)

                if card_num_val != 13:
                    actions.append(Card(suit, self.num_to_Enum(card_num_val)))

                elif self.field_cards[n][11] == 1 and not is_king_out:
                    actions.append(Card(suit, Number.KING))

            if is_ace_out and not is_king_out:
                actions.append(Card(suit, Number.KING))

            elif is_king_out and not is_ace_out:
                actions.append(Card(suit, Number.ACE))

        return list(set(actions))


    # 自分が出せる手のリスト取得
    def my_actions(self):
        actions = []
        for legal in self.legal_actions():
            if self.players_cards[self.turn_player].check(legal)==True:
                actions.append(legal)
        return actions
    def my_actions_str(self):
        actions = []
        for legal in self.legal_actions():
            if self.players_cards[self.turn_player].check(legal)==True:
                actions.append(legal)
        return [str(i) for i in actions]

    # 自分の手札取得
    def my_hands(self):
        return self.players_cards[self.turn_player]
    def my_hands_str(self):
        return [str(i) for i in self.players_cards[self.turn_player]]


    def num_to_Enum(self,num):
        enum_list=[Number.ACE,Number.TWO,Number.THREE,Number.FOUR,
                   Number.FIVE,Number.SIX,Number.SEVEN,Number.EIGHT,
                   Number.NINE,Number.TEN,Number.JACK,Number.QUEEN,
                   Number.KING]
        return enum_list[num-1]


    # 次の状態の取得
    def next(self,action,pass_flag = 0):
        if self.my_actions()==[]:
            self.pass_count[self.turn_player]+=1
            self.pass_check()
        elif pass_flag == 1:
            self.pass_count[self.turn_player]+=1
            self.pass_check()
        else:
            self.players_cards[self.turn_player].remove(action)
            self.put_card(action)
            self.all_actions.append(action)
            self.all_players.append(self.turn_player)

        #次のプレイヤーに
        self.next_player()
        return State(players_num=self.players_num,field_cards=self.field_cards, players_cards=self.players_cards,turn_player=self.turn_player,pass_count=self.pass_count,out_player=self.out_player,all_actions=self.all_actions,all_players=self.all_players)

    #次のプレイヤーの取得
    def next_player(self):
        flag=0
        while flag==0:
            self.turn_player = (self.turn_player + 1) % self.players_num

            if self.turn_player not in self.out_player:
                flag=1

    #パスの上限判定
    def pass_check(self):
        out_list=self.out_player
        if self.pass_count[self.turn_player]>3:
            for card in self.my_hands():
                self.put_card(card)
                self.all_actions.append(card)
                self.all_players.append(self.turn_player)

            out_list.append(self.turn_player)

            self.out_player=out_list

    def to_str(self,num):
        return str(num)

    #勝ち負け判定
    def is_done(self):
        return len(self.my_hands())==0


    # 状態表示
    def __str__(self):
        str = ''
        field_cards=self.field_cards.tolist()
        out_list=[list(map(mul,self.all_cards[i],field_cards[i])) for i in range(4)]
        str += "場のカード\n\n"
        for i in range(len(out_list)):
            minilist=out_list[i]
            for j in range(len(minilist)):
                if minilist[j] == "":
                    str += " -- "
                else:
                    str +=" "+minilist[j]+" "
            str += '\n'
        num=self.to_str(self.turn_player)
        pass_cnt=self.to_str(self.pass_count[self.turn_player])
        str+="\nプレイヤー"+num+"番　　パス回数"+pass_cnt+"\n"
        str += "\nあなたの手札\n"

        out_list=self.my_hands_str()
        for i in range(len(out_list)):
            str+=out_list[i]
            str+=" "

        str += "\n\n出せるカード\n"

        out_list=self.my_actions_str()
        for i in range(len(out_list)):
            str+=out_list[i]
            str+=" "

        str += "\n"

        return str

In [ ]:
# @title
def num_to_Card(number,suit):
    number_list=[Number.ACE,Number.TWO,Number.THREE,Number.FOUR,
                Number.FIVE,Number.SIX,Number.SEVEN,Number.EIGHT,
                Number.NINE,Number.TEN,Number.JACK,Number.QUEEN,
                Number.KING]
    suit_list=[Suit.SPADE,Suit.CLUB,Suit.HEART,Suit.DIAMOND]
    return Card(suit_list[suit],number_list[number-1])

In [ ]:
# ランダム行動 AI
def random_action(state):
    my_actions = state.my_actions()
    if my_actions != []:
        return my_actions[random.randint(0, len(my_actions)-1)]
    else:
        my_actions=[]
    return my_actions

# **※課題 my_AIの作成**

`return (出したいカード),(パスを行うか)`を行うAIを作成してください

In [ ]:
MY_PLAYER_NUM = 0
import random

def my_AI(state):
    # --- 調整用パラメータ (戦略を尖らせるための重み) ---
    params = {
        'W_CIRCULAR_DIST': 8,    # 7からの距離（端に近いほど出しにくい）
        'W_MY_PATH': 16,         # 自分の次の手に繋がるボーナス
        'W_OTHERS_RISK': -78,    # 他人に塩を送るリスク（深度1あたりの減点）
        'W_SUIT_DOM': 17,        # スート支配力の重み
        'W_WIN_DASH': 293,       # 勝ち圏内の放出意欲
        'P_THRESHOLD_BASE': -25,  # 基本のパスしきい値
        'P_KILL_ZONE': 158,      # 相手をハメる時のパスしきい値
        'P_WIN_THRESHOLD': 9     # 勝ち圏内のパスしきい値
    }

    my_actions = state.my_actions()
    me = state.turn_player
    my_hands = state.my_hands()
    pass_used = state.pass_count[me]

    if not my_actions:
        return None, 1

    suit_to_index = {'♠':0, '♣':1, '♡':2, '♢':3}
    opp_pass_left = [3 - state.pass_count[i] for i in range(len(state.players_cards)) if i != me]
    opp_min_pass = min(opp_pass_left) if opp_pass_left else 3
    opp_min_hand = min(len(h) for i, h in enumerate(state.players_cards) if i != me) if state.players_cards else 13
    my_hand_indices = set((suit_to_index[c.suit.value], c.number.val - 1) for c in my_hands)

    def get_chain_risk(suit_idx, start_num, direction):
        """
        指定した方向に、自分が持っていないカードが何枚連続しているか(他人のための道)を計算
        """
        risk_count = 0
        curr = (start_num + direction) % 13
        # トンネルがあるので最大6枚まで探索（7の反対側まで）
        for _ in range(6):
            if state.field_cards[suit_idx][curr] == 1: # すでに出ている
                break
            if (suit_idx, curr) in my_hand_indices: # 自分で止めている
                break
            # 誰かが持っているはずのカード
            risk_count += 1
            curr = (curr + direction) % 13
        return risk_count

    best_score = -1e9
    best_cards = []

    for card in my_actions:
        s = card.suit.value
        i = suit_to_index[s]
        n = card.number.val - 1
        score = 0

        # 1. 基本：円環距離 (端のカードほど保持価値が高い)
        dist_from_7 = abs(n - 6)
        circular_dist = min(dist_from_7, 13 - dist_from_7)
        score += circular_dist * params['W_CIRCULAR_DIST']

        # 2. 深度ベースの開放リスクと自己利益
        # 隣接する2方向（トンネル含む）を確認
        for direction in [1, -1]:
            neighbor = (n + direction) % 13
            # その方向がまだ未開放の場合のみ評価
            if state.field_cards[i][neighbor] == 0:
                if (i, neighbor) in my_hand_indices:
                    # 自分が持っているなら「自分の道」
                    score += params['W_MY_PATH']
                else:
                    # 自分が持っていないなら「他人の道」を何枚分開けてしまうか
                    risk_depth = get_chain_risk(i, n, direction)
                    score += risk_depth * params['W_OTHERS_RISK']

        # 3. スート支配 (そのマークを多く持っているなら、そのマークを優先的に進める)
        same_suit_count = sum(1 for c in my_hands if c.suit.value == s)
        score += same_suit_count * params['W_SUIT_DOM']

        # 4. 終盤・状況補正
        if len(my_hands) <= opp_min_hand:
            score += params['W_WIN_DASH']

        # 特殊ルール補正: トンネルの端（1, 13）を出す際、逆側の状況を見る
        if n == 0 or n == 12:
            opposite = 12 if n == 0 else 0
            if state.field_cards[i][opposite] == 0 and (i, opposite) not in my_hand_indices:
                # 逆側のカードを自分が持っていないのにトンネルを開けるのは非常に危険
                score -= 50

        if score > best_score:
            best_score = score
            best_cards = [card]
        elif abs(score - best_score) < 1e-9:
            best_cards.append(card)

    chosen_card = random.choice(best_cards)

    # --- 戦略的パスの動的判断 ---
    pass_threshold = params['P_THRESHOLD_BASE']

    # 相手のパスが尽きそうな時は、より「出さない」選択を強化
    if opp_min_pass == 0:
        pass_threshold = params['P_KILL_ZONE']
    elif opp_min_pass == 1:
        pass_threshold = params['P_KILL_ZONE'] * 0.7

    # 自分のパス残弾が少ない時は、評価が低くても出す（自滅回避）
    if (3 - pass_used) <= 1:
        pass_threshold = -9999

    # 自分が勝てそうな時は、変に止めずに流す
    if len(my_hands) <= opp_min_hand and best_score > 0:
        pass_threshold = params['P_WIN_THRESHOLD']

    # 評価値がしきい値を下回るなら、戦略的パスを選択
    if best_score < pass_threshold and pass_used < 3:
        return None, 1

    return chosen_card, 0

**ランダムAIとの勝率チェック**


In [ ]:
# パラメータ
EP_GAME_COUNT = 500  # 1評価あたりのゲーム数

def player_point(ended_state):
    if ended_state.turn_player == MY_PLAYER_NUM:
        return 1
    return 0

def play(next_actions):
    state = State()
    while True:
        if state.is_done():
            break
        pass_flag=0
        if state.turn_player == MY_PLAYER_NUM:
            action,pass_flag = my_AI(state)
        else:
            action = random_action(state)
        # 次の状態の取得
        if pass_flag == 1:
            state = state.next(action,pass_flag)
        else:
            state = state.next(action)
    return player_point(state)

# 任意のアルゴリズムの評価
def evaluate_algorithm_of(label, next_actions):
    # 複数回の対戦を繰り返す
    total_point = 0
    for i in range(EP_GAME_COUNT):
        total_point += play(next_actions)
        print('\rEvaluate {}/{}'.format(i + 1, EP_GAME_COUNT), end='')
    print('')

    # 平均ポイントの計算
    average_point = total_point / EP_GAME_COUNT
    print(label.format(average_point))

# VSランダム
next_actions = (random_action, random_action)
evaluate_algorithm_of('VS_Random {:.3f}', next_actions)

Evaluate 500/500
VS_Random 0.818


In [ ]:
import optuna
import random

# ==========================================
# 1. AIロジック (パラメータ p を受け取る)
# ==========================================
def my_AI(state, p):
    my_actions = state.my_actions()
    if not my_actions:
        return None, 1

    me = state.turn_player
    my_hands = state.my_hands()
    pass_used = state.pass_count[me]
    suit_to_index = {'♠':0, '♣':1, '♡':2, '♢':3}

    # 状況把握
    opp_pass_left = [3 - state.pass_count[i] for i in range(len(state.players_cards)) if i != me]
    opp_min_pass = min(opp_pass_left) if opp_pass_left else 3
    opp_min_hand = min(len(h) for i, h in enumerate(state.players_cards) if i != me)
    my_hand_indices = set((suit_to_index[c.suit.value], c.number.val - 1) for c in my_hands)

    def get_chain_depth(suit_idx, start_num, direction):
        """敵に何枚の道を開けてしまうか(支配深度)を計算"""
        depth = 0
        curr = (start_num + direction) % 13
        for _ in range(6):
            if state.field_cards[suit_idx][curr] == 1: break # すでに出ている
            if (suit_idx, curr) in my_hand_indices: break    # 自分で止めている
            depth += 1
            curr = (curr + direction) % 13
        return depth

    best_score = -1e9
    best_cards = []

    for card in my_actions:
        s = card.suit.value
        i = suit_to_index[s]
        n = card.number.val - 1
        score = 0

        # ① 円環距離
        d = abs(n - 6)
        score += min(d, 13 - d) * p['W_CIRCULAR_DIST']

        # ② 隣接リスク・ベネフィット (支配深度対応)
        for direction in [1, -1]:
            neighbor = (n + direction) % 13
            if state.field_cards[i][neighbor] == 0:
                if (i, neighbor) in my_hand_indices:
                    score += p['W_MY_PATH']
                else:
                    depth = get_chain_depth(i, n, direction)
                    score += depth * p['W_CHAIN_RISK']

        # ③ スート支配 (そのマークを何枚持っているか)
        score += sum(1 for c in my_hands if c.suit.value == s) * p['W_SUIT_DOM']

        # ④ 終盤調整
        if len(my_hands) <= opp_min_hand:
            score += p['W_WIN_DASH']

        # ⑤ トンネルリスク (端を出すとき逆側を持っていないなら大幅減点)
        if n == 0 or n == 12:
            opposite = 12 if n == 0 else 0
            if state.field_cards[i][opposite] == 0 and (i, opposite) not in my_hand_indices:
                score += p['W_TUNNEL_RISK']

        if score > best_score:
            best_score = score
            best_cards = [card]
        elif abs(score - best_score) < 1e-9:
            best_cards.append(card)

    chosen_card = random.choice(best_cards)

    # --- 戦略的パスの判断 ---
    th = p['P_THRESHOLD_BASE']
    if opp_min_pass == 0:
        th = p['P_KILL_ZONE'] # 誰かのパスが切れたらハメモード

    # 自分のパス残り1回なら強制的に出す(自滅回避)
    if (3 - pass_used) <= 1:
        th = -99999

    # 自分が上がりそうならパスせず出す
    if len(my_hands) <= opp_min_hand and best_score > 0:
        th = p['P_WIN_THRESHOLD']

    if best_score < th and pass_used < 3:
        return None, 1

    return chosen_card, 0

# ==========================================
# 2. ゲーム実行部 (play 関数)
# ==========================================
def play(params):
    # ここで State() を初期化 (お使いのクラス名に合わせてください)
    state = State()

    MY_PLAYER_NUM = 0 # 自分のプレイヤー番号

    while not state.is_done():
        pass_flag = 0
        if state.turn_player == MY_PLAYER_NUM:
            # 自分のAIにパラメータを渡して実行
            action, pass_flag = my_AI(state, params)
        else:
            # 相手(ランダム)の行動
            action = random_action(state)

        if pass_flag == 1:
            state = state.next(action, pass_flag)
        else:
            state = state.next(action)

    # 自分が1位(turn_playerが自分のまま終了)なら1を返す
    # ※Stateクラスの仕様に合わせて勝利判定を修正してください
    if state.turn_player == MY_PLAYER_NUM:
        return 1
    return 0

# ==========================================
# 3. Optuna 目的関数
# ==========================================
def objective(trial):
    # パラメータの探索範囲を定義
    params = {
        'W_CIRCULAR_DIST':  trial.suggest_int('W_CIRCULAR_DIST', -15, 45),
        'W_MY_PATH':        trial.suggest_int('W_MY_PATH', -35, 145),
        'W_CHAIN_RISK':     trial.suggest_int('W_CHAIN_RISK', -290, 70),
        'W_SUIT_DOM':       trial.suggest_int('W_SUIT_DOM', -50, 150),
        'W_WIN_DASH':       trial.suggest_int('W_WIN_DASH', -125, 475),
        'W_TUNNEL_RISK':    trial.suggest_int('W_TUNNEL_RISK', -300, 100),
        'P_THRESHOLD_BASE': trial.suggest_int('P_THRESHOLD_BASE', -115, 145),
        'P_KILL_ZONE':      trial.suggest_int('P_KILL_ZONE', -50, 350),
        'P_WIN_THRESHOLD':  trial.suggest_int('P_WIN_THRESHOLD', -100, 100)
    }

    total_win = 0
    EP_GAME_COUNT = 5000 # 1回の試行につき1000試合で評価

    for _ in range(EP_GAME_COUNT):
        total_win += play(params)

    return total_win / EP_GAME_COUNT

# ==========================================
# 4. 実行メイン
# ==========================================
if __name__ == '__main__':
    # win_rateを最大化するスタディを作成
    # storage を設定すると中断しても再開できますが、まずはメモリ上で実行
    study = optuna.create_study(direction='maximize')

    print("最適化を開始します。n_trials=100 (合計10万試合)...")
    study.optimize(objective, n_trials=1000)

    print("\n" + "="*30)
    print("【最適化完了】")
    print(f"最高勝率: {study.best_value:.4f}")
    print(f"最高パラメータ: {study.best_params}")
    print("="*30)

[I 2026-01-18 14:19:04,723] A new study created in memory with name: no-name-0adf80be-6be9-4fc9-91e7-60f6080b5991


最適化を開始します。n_trials=100 (合計10万試合)...


[I 2026-01-18 14:19:45,156] Trial 0 finished with value: 0.1724 and parameters: {'W_CIRCULAR_DIST': -3, 'W_MY_PATH': 89, 'W_CHAIN_RISK': 28, 'W_SUIT_DOM': 79, 'W_WIN_DASH': 185, 'W_TUNNEL_RISK': 33, 'P_THRESHOLD_BASE': 126, 'P_KILL_ZONE': 51, 'P_WIN_THRESHOLD': 10}. Best is trial 0 with value: 0.1724.
[I 2026-01-18 14:20:21,637] Trial 1 finished with value: 0.8474 and parameters: {'W_CIRCULAR_DIST': 14, 'W_MY_PATH': 115, 'W_CHAIN_RISK': -65, 'W_SUIT_DOM': 38, 'W_WIN_DASH': -77, 'W_TUNNEL_RISK': 18, 'P_THRESHOLD_BASE': 92, 'P_KILL_ZONE': 91, 'P_WIN_THRESHOLD': 75}. Best is trial 1 with value: 0.8474.
[I 2026-01-18 14:20:58,224] Trial 2 finished with value: 0.8262 and parameters: {'W_CIRCULAR_DIST': 2, 'W_MY_PATH': 103, 'W_CHAIN_RISK': -156, 'W_SUIT_DOM': 107, 'W_WIN_DASH': 409, 'W_TUNNEL_RISK': -160, 'P_THRESHOLD_BASE': -48, 'P_KILL_ZONE': 314, 'P_WIN_THRESHOLD': 99}. Best is trial 1 with value: 0.8474.
[I 2026-01-18 14:21:32,827] Trial 3 finished with value: 0.7796 and parameters: {'W_

In [ ]:
import random

# --- 1. 最適化対象のパラメータ範囲を定義 ---
def generate_random_params():
    return {
        'W_CIRCULAR_DIST':  random.randint(0, 10),   # 7からの距離
        'W_MY_PATH':        random.randint(10, 50),  # 自分の道
        'W_CHAIN_RISK':     random.randint(-60, -20),# 敵に道を開けるリスク(深度1あたり)
        'W_SUIT_DOM':       random.randint(5, 40),   # スート支配力
        'W_WIN_DASH':       random.randint(50, 120), # 終盤の放出
        'W_TUNNEL_RISK':    random.randint(-80, -20),# 逆側を持ってないトンネル開放リスク
        'P_THRESHOLD_BASE': random.randint(0, 30),   # 基本パス
        'P_KILL_ZONE':      random.randint(80, 130), # 相手をハメる時のパス
        'P_WIN_THRESHOLD':  random.randint(-10, 30)  # 勝ち圏パス
    }

# --- 2. 強化版AI (支配深度ロジック搭載) ---
def my_AI(state, p):
    my_actions = state.my_actions()
    if not my_actions:
        return None, 1

    me = state.turn_player
    my_hands = state.my_hands()
    pass_used = state.pass_count[me]
    suit_to_index = {'♠':0, '♣':1, '♡':2, '♢':3}

    opp_pass_left = [3 - state.pass_count[i] for i in range(len(state.players_cards)) if i != me]
    opp_min_pass = min(opp_pass_left) if opp_pass_left else 3
    opp_min_hand = min(len(h) for i, h in enumerate(state.players_cards) if i != me)
    my_hand_indices = set((suit_to_index[c.suit.value], c.number.val - 1) for c in my_hands)

    # 敵に何枚の道を開けてしまうか計算する補助関数
    def get_chain_depth(suit_idx, start_num, direction):
        depth = 0
        curr = (start_num + direction) % 13
        for _ in range(6): # 最大6枚先まで
            if state.field_cards[suit_idx][curr] == 1: break # すでに出ている
            if (suit_idx, curr) in my_hand_indices: break    # 自分が持っている(止まる)
            depth += 1
            curr = (curr + direction) % 13
        return depth

    best_score = -1e9
    best_cards = []

    for card in my_actions:
        s = card.suit.value
        i = suit_to_index[s]
        n = card.number.val - 1
        score = 0

        # 円環距離
        d = abs(n - 6)
        score += min(d, 13 - d) * p['W_CIRCULAR_DIST']

        # 隣接リスク・ベネフィット (支配深度対応)
        for direction in [1, -1]:
            neighbor = (n + direction) % 13
            if state.field_cards[i][neighbor] == 0:
                if (i, neighbor) in my_hand_indices:
                    score += p['W_MY_PATH']
                else:
                    # 深度に応じた減点
                    depth = get_chain_depth(i, n, direction)
                    score += depth * p['W_CHAIN_RISK']

        # スート支配
        score += sum(1 for c in my_hands if c.suit.value == s) * p['W_SUIT_DOM']

        # 終盤調整
        if len(my_hands) <= opp_min_hand:
            score += p['W_WIN_DASH']

        # トンネル端点のリスク管理
        if n == 0 or n == 12:
            opposite = 12 if n == 0 else 0
            if state.field_cards[i][opposite] == 0 and (i, opposite) not in my_hand_indices:
                score += p['W_TUNNEL_RISK']

        if score > best_score:
            best_score = score
            best_cards = [card]
        elif abs(score - best_score) < 1e-9:
            best_cards.append(card)

    chosen_card = random.choice(best_cards)

    # パス判断
    th = p['P_THRESHOLD_BASE']
    if opp_min_pass == 0: th = p['P_KILL_ZONE']
    if (3 - pass_used) <= 1: th = -9999
    if len(my_hands) <= opp_min_hand and best_score > 0: th = p['P_WIN_THRESHOLD']

    if best_score < th and pass_used < 3:
        return None, 1
    return chosen_card, 0

# --- 3. シミュレーション実行 (既存の枠組みを流用) ---
# ... (play, evaluate_params 関数はあなたの既存のものをそのまま使用) ...

if __name__ == '__main__':
    best_win_rate = 0.76  # 現在の目標値
    print(f"Starting Enhanced Search... Target > {best_win_rate*100}%")

    while True:
        current_params = generate_random_params()
        win_rate = evaluate_params(current_params)

        print(f'\nResult: {win_rate:.4f} (Params: {current_params})')

        if win_rate > best_win_rate:
            best_win_rate = win_rate
            print("*" * 50)
            print(f"!!! NEW RECORD: {win_rate*100:.2f}% !!!")
            print(f"Best Params: {current_params}")
            print("*" * 50)

In [ ]:
# ランダムAIと対戦
state = State()
turn = 0

print("----------- ゲーム開始 -----------\nmy_AI :プレイヤー"+str(MY_PLAYER_NUM)+"番")

# ゲーム終了までのループ
while True:
    turn += 1
    num = state.turn_player

    # ゲーム終了時
    if state.is_done():
        print("------- ゲーム終了　ターン",turn,"-------")
        print("* 勝者 プレイヤー"+str(num)+"番")
        break;
    else:
        print("------------ ターン",turn,"------------")

    pass_flag = 0
    # 行動の取得
    if num == MY_PLAYER_NUM:
        action,pass_flag = my_AI(state)
        print(termcolor.colored(state, 'red'))
    else:
        action = random_action(state)
        print(state)

    print("出したカード")
    if state.my_actions() == [] or pass_flag == 1:
        print("パス")
        if state.pass_count[num] >= 3:
          print("\n* プレイヤー"+str(num)+"番 バースト")
    else:
        print(action)

    # 次の状態の取得
    if pass_flag == 1:
        state = state.next(action,pass_flag)
    else:
        state = state.next(action)

### 1.手札に関する関数

In [ ]:
 #リストで手札を表示する
state.my_hands()

In [ ]:
#リストで手札の数字を表示する
state.my_hands().check_number()

In [ ]:
#リストで手札のマークを表示する
state.my_hands().check_suit()

In [ ]:
#リストで自分が出せるカードを表示する
state.my_actions()

In [ ]:
#リストで自分が出せるカードの数字を表示する
Hand(state.my_actions()).check_number()

In [ ]:
#リストで自分が出せるカードの記号を表示する
Hand(state.my_actions()).check_suit()

### 2.場の札に関する関数

In [ ]:
#場のカードを表示する
state.field_cards

In [ ]:
#場で出せるカードをリストで取得する
state.legal_actions()

In [ ]:
#場で出せるカードの数字をリストで取得する
Hand(state.legal_actions()).check_number()

In [ ]:
#場で出せるカードの記号をリストで取得する
Hand(state.legal_actions()).check_suit()

 ### 3.状態に関する関数

In [ ]:
#今のプレイヤーの番号を表示する
state.turn_player

In [ ]:
#3回パスをしてしまったプレイヤーを表示する
state.out_player

### 4.pythonプログラミングの基礎

#### print

何かを表示するときはprintというものを使います。

In [ ]:
print("Hello World")
print(5)

#### 計算式

四則演算ができます。

In [ ]:
print(5+4)  #足し算
print(5-4)  #引き算
print(5*4)  #掛け算
print(5/4)  #割り算
print(5%4)  #割った余りを求める

#### 変数

数学の文字と同じで数字を代入することができます。文章も代入できます。

In [ ]:
a = 3
b = 1+3  #計算式の形で代入ができます
c = "こんにちは"  #文字列も代入できます
aisatsu = "こんばんは"  #変数名は何文字でもいいです
print(a)
print(b)
print(c)
print(aisatsu)

#### リスト

リストを使うとたくさんの数字や文字をまとめることができます。数学の添字と同じです。

In [ ]:
l = [1,2,3,4,5]
print(l[3])  #0番目から数えて3番目の要素を返します

l.append(100)  #末尾に100を追加します
print(l)

#### if文

if文を使うことで条件分岐をすることができます。

#### for文

for文を使うことで繰り返し処理をすることができます。

In [ ]:
#:とインデントを忘れないようにしてください。
for i in range(5):
    print("Hello World.")

こんな使い方もあります。

In [ ]:
for item in ["Apple", "Orange", "Banana", "Melon"]:
    print(item)

#### 関数

関数というものを使うと何度も使う機能を少ないコーディングで書くことができます。数学の関数と同じで値を入力すると値を計算して返します。

In [ ]:
#:とインデントを忘れないようにしてください。
#引数を3乗する関数
def testfunc(hikisu):
    cube = hikisu*hikisu*hikisu
    return cube

In [ ]:
print(testfunc(3))

### 4.便利な関数

In [ ]:
#リストを定義する
l = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [ ]:
#最大値を求める関数
max(l)

In [ ]:
#最小値を求める関数
min(l)

In [ ]:
#昇順に並び替える関数
sorted(l)

In [ ]:
#降順に並び替えるときはreverse = Trueをつける
sorted(l, reverse = True)

In [ ]:
#7からの距離を求める関数
def DistFrom7(num):
    return abs(num-7)

In [ ]:
#keyを使うと7からの距離で最大最小並び替えができる
print(max(l, key = DistFrom7))
print(min(l, key = DistFrom7))
print(sorted(l, key = DistFrom7))

In [ ]:
#使用例
#手札を7から近い順に並び替える
def DistFrom7(hand):
    return abs(hand.number.val-7)


hands_sorted = sorted(state.my_hands(), key = DistFrom7, reverse = False)
print(state.my_hands())
print(hands_sorted)

### 5.Classの説明
この7ならべプログラムではclassと呼ばれる概念が使われています。classを使うと「もの」をわかりやすく記述することができます。

たとえば、身長と好きな色がある「人」というクラスを作ってみます。

In [ ]:
class person:
    height = 0
    favcolor = "hoge"

これで「人」が定義できました。それでは田中さんを作ってみます。

In [ ]:
tanaka = person()

tanaka.heightとすると身長が、tanaka.colorとすると好きな色が表示できます。しかし田中さんの身長と好きな色はまだ初期のままです。

In [ ]:
print(tanaka.height)
print(tanaka.favcolor)

田中さんの身長と好きな色を代入してみましょう

In [ ]:
tanaka.height=150
tanaka.favcolor="blue"

In [ ]:
print(tanaka.height)
print(tanaka.favcolor)

classの中には関数を入れることもできます。ためしに身長と好きな色を表示する関数を作ってみます。<br>関数内で変数を扱うときは「そのクラス自身の変数」であることをいうためにself.heightのようにします。

In [ ]:
class person:
    height = 0
    favcolor = "hoge"

    def explain(self):
        print("身長は"+str(self.height)+"、好きな色は"+self.favcolor)

In [ ]:
okada = person()
okada.height = 160
okada.favcolor = "pink"

In [ ]:
okada.explain()

\_\_init\_\_という関数はclassを代入したときに自動的に動く関数です。これを使うと最初に変数を代入するときなどに便利です。

In [ ]:
class person:
    def __init__(self, height, favcolor):
        self.height = height
        self.favcolor = favcolor

    def explain(self):
        print("身長は"+str(self.height)+"、色は"+self.favcolor)

In [ ]:
suzuki = person(170, "yellow")

In [ ]:
suzuki.explain()